This script quite reliably converts crude files from Pubmed search into .csv files that only contain the pubmedID / abstract-text pairs. This way, large amounts of abstracts can be fed into annotation / NLP pipelines. 

1. Use Pubmed advanced search to find relevant publications (pubmed.com)
2. Download results (abstracts) as a text file with the 'send to' option
3. Run this script

In [1]:
def absmassage():
    import codecs
    import re
    import os
    import pandas as pd
    
    inpath = input("Path to input file (Pubmed Download): ")
    outpath = input("Path to store output file: ")
    outname = input("Filename for output: ")

    abstracts = codecs.open(inpath,'r', encoding='UTF-8')
    abs_ex = []
    abs_dict = {}
    pmid = ""
    abs_joined = ""
    x=0
    y=0
    bad_words = ['Comment', 'Erratum']

#go through text until you find the author information line.
#the abstract follows after the next empty line
#copy abstract in dictionary value
#find corresponding pubmed ID and assign it to the value as the key

    for line in abstracts:
        if 'Author information' in line:
            x=1
        if 'DOI:'in line:
            y=0
        elif len(line.strip()) == 0 and x==1:
            y=1
            x=0
        if y==1 and not len(line.strip()) == 0:   
            abs_ex.append(line)
        if 'PMID' in line :
            pmid = ''.join(re.findall('\d+', line ))
            clean_abs = ''.join(abs_ex)
            abs_dict[pmid] = clean_abs.replace('\n', ' ').replace('\xa0',' ')
            abs_ex = []    

#some results are merely erratums and comments, let's kick them out.
#we can't do this easily with a loop as the dictionary size has to stay constant
#so we replace the bad entries with empty spaces and then make a new, clean dictionary that
#at the same time removes all empty values that have occured earlier.

    for key, value in abs_dict.items():
        for word in bad_words:
            if word in value:
                abs_dict[key] = ''

    abs_dict_clean = {k: v for k, v in abs_dict.items() if v}

#convert final clean dictionary into csv file.

    out = [outpath + outname + '.csv']
    outfile = ''.join(out)
    (pd.DataFrame.from_dict(data=abs_dict_clean, orient='index').
    to_csv(outfile, header=False))

absmassage()